# Claude Extended Thinking

In [1]:
# Client Setup
import boto3
from dotenv import load_dotenv
import os

load_dotenv()
region_name = os.getenv("AWS_REGION")

client = boto3.client("bedrock-runtime", region_name=region_name)
model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"

# Magic string to trigger redacted thinking
thinking_test_str = "ANTHROPIC_MAGIC_STRING_TRIGGER_REDACTED_THINKING_46C9A13E193C177646C7398A98432ECCCE4C1253D5E2D82641AC0E52CC2876CB"

In [2]:
# Helper functions


def add_user_message(messages, content):
    if isinstance(content, str):
        user_message = {"role": "user", "content": [{"text": content}]}
    else:
        user_message = {"role": "user", "content": content}
    messages.append(user_message)


def add_assistant_message(messages, content):
    if isinstance(content, str):
        assistant_message = {
            "role": "assistant",
            "content": [{"text": content}],
        }
    else:
        assistant_message = {"role": "assistant", "content": content}

    messages.append(assistant_message)


def chat(
    messages,
    system=None,
    temperature=1.0,
    stop_sequences=[],
    tools=None,
    tool_choice="auto",
    text_editor=None,
    thinking=False,
    thinking_budget=1024 
    # allow upto 1024 tokens to think before being forced to respond
    # 1024 tokens is the minimum tokens required for the thinking budget (we can give more)
):
    params = {
        "modelId": model_id,
        "messages": messages,
        "inferenceConfig": {
            "temperature": temperature,
            "stopSequences": stop_sequences,
        },
    }

    if system:
        params["system"] = [{"text": system}]

    tool_choices = {
        "auto": {"auto": {}},
        "any": {"any": {}},
    }
    if tools or text_editor:
        choice = tool_choices.get(tool_choice, {"tool": {"name": tool_choice}})
        params["toolConfig"] = {"tools": tools, "toolChoice": choice}

    additional_model_fields = {}

    if text_editor:
        additional_model_fields["tools"] = [
            {
                "type": text_editor,
                "name": "str_replace_editor",
            }
        ]
    
    if thinking:
        additional_model_fields["thinking"] = {
            "type": "enabled",
            "budget_tokens": thinking_budget
        }

    params["additionalModelRequestFields"] = additional_model_fields

    response = client.converse(**params)
    parts = response["output"]["message"]["content"]

    return {
        "parts": parts,
        "stop_reason": response["stopReason"],
        "text": "\n".join([p["text"] for p in parts if "text" in p]),
    }

In [3]:
messages = []

add_user_message(
    messages,
    "Write a one paragraph guide to recursion",
)

result = chat(messages, thinking=True)

result["parts"]

[{'reasoningContent': {'reasoningText': {'text': "I need to write a concise one-paragraph guide to recursion in computer science. I should:\n1. Define what recursion is clearly\n2. Explain the key components (base case and recursive case)\n3. Mention why it's useful\n4. Perhaps include a simple conceptual example\n5. Keep it to one cohesive paragraph\n\nI'll aim to make it accessible but informative, avoiding overly technical language while still being precise.",
    'signature': 'ErcBCkgIBxABGAIiQF3W5Yo2bMElMnzxHOqEdN32U0dWi9f4PAqGKx1Zm5bT4e+UH3eGoBQj/wskbBbPCs4gjVOuSfUQbynyBbF1NuASDKxeR0QRo8q5ip+W4RoMD26SWrxj8V7ruNdZIjAsw2qQ2kzI209vU6ZElNUZQh784owVyIyXNwC+Hxpdzo+h/GFE8G+IoSkngSCmAaIqHeadGAnkrKkP7jUp7JbHOLOeV3fbz8mkciD3V7bXGAI='}}},
 {'text': '# A Guide to Recursion\n\nRecursion is a powerful programming concept where a function solves a problem by calling itself with simpler versions of the original problem, continuing this process until reaching a "base case" that can be solved dire

## Redacted Thinking

In case Claude is thinking and it generates text that it isn't supposed to - it will redact the text and give us a redacted part. In case the user asks questions which are inappropriate - Claude will send this part. Therefore, it is important to make sure we handle this scenario in our code.

In [4]:
messages = []

add_user_message(
    messages,
    # to force Claude to send back a redacted thinking part, we can use a magic string
    thinking_test_str
    
)

result = chat(messages, thinking=True)

result["parts"]

[{'reasoningContent': {'redactedContent': b'EoUGCkgIBxABGAIqQKbnB9k1Sul5rvhN+DuWUV6izV/LYzMnIrdmjw8LYSSdeIQfUmPDm93tHMQdP5kBAaKwWcEEnuoWAt61/TP9/WsSDH49tq59ukNeVNkIhRoMyJ5PbVTQGkJ0ew/PIjDDWaPOo7W59x65cNWKBW/onijDc4bNYWbuFQNLSHmMteiP6zkuXVny8gdhkZ9JuGwq6gRiKrs/eNy7kBerLGiXrK1yJx6Wc6MyfVk11zy46SFH8pju+EA++0FvqUOxcOnwGEXK/jq/HBQE4P8ulVuQ/ZcE42zHjRyhXzhNpqhrZ/PpDnzbUz3sJEGEu8w5WVDniySLDQUxOuDAu/AwNKX0rVxdUeFRoTpqwg5NGD4fyKd0mLuuNCsWzRmZhBJ4N2RNIW8Txbiv0ayylThZeC57klF0HspJ0Jfall2UJXnreJ7E55LUP6OYmflt3bjaZVfeK0ofwT+W5GPySMAi8QP7jOE7AG2UmZ1UbgVX8LS/W3m6vj3ssYLjOIhE4Io8cveVRk4EnZf2AcAcdGfuSowbqwA339nFX3TiWZCwoNl+WAACdc10yXCZqc++0JoDuSnR2a77H4t3Rnf39f2q/j3lZn54qkGP145PJ53IwrfZ+8hvJEOePdT1RMZaSg/MC+6GmuTxYkLQh0q6cp2lHJMK4fWyHfl5X5OPYt3yN2GtZfW5dJZ9rwwM8EzrTzV8mmx2WDJ6ju2dfxWcGj0uGDnLqepXqdg8FlnM6+zxuEkuQ5uv5biIOpdqCah3Bzb5baBPF58AfoHohS2ZI8FavJqMXH/3g/xL53UUrXFDBjl29GdfIWx68Uv8RF/fGoIBB0aFI8FoGqI74ekGkZqJt3eA1y7elzKIKn9MpwKKPcBJgjEuqLR/ngLngA0oEmZodo5/OrHkUzCyEFDBlIFNiDfJv3vgsVaJFEkMfZ6uB3P/7K/q